# Regression test suite for the IMAGENATOR service:

This notebook runs a suite of regression tests against the Harmony IMAGENATOR Service chain. These tests use a sample TEMPO NO₂ granule to verify that IMAGENATOR logic works as expected and that output matches reference data in `reference_data/`.


## Prerequisites

- Create the `papermill-imagenator` environment:
  ```bash
  conda env create -f ./environment.yaml && conda activate papermill-imagenator
  ```
- Ensure a `.netrc` file with your Earthdata Login credentials is present here.

## Import required packages:

In [ ]:
import sys

sys.path.append('../shared_utils')
from utilities import print_success
from pathlib import Path
from tempfile import TemporaryDirectory

from harmony import Client, Collection, Environment, Request

## Set default parameters:

`papermill` requires default values for parameters used on the workflow. In this case, `harmony_host_url`.

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

### Identify Harmony environment (for easier reference):

In [ ]:
host_environment = {
    'http://localhost:3000': Environment.LOCAL,
    'https://harmony.sit.earthdata.nasa.gov': Environment.SIT,
    'https://harmony.uat.earthdata.nasa.gov': Environment.UAT,
    'https://harmony.earthdata.nasa.gov': Environment.PROD,
}

harmony_environment = host_environment.get(harmony_host_url)

if harmony_environment is not None:
    harmony_client = Client(env=harmony_environment)

# Begin regression tests:

The IMAGENATOR is currently only configured against collections in the UAT environment.

### Set up environment-dependent variables for imagenator L2:

Define the collection and granule for testing in each environment.

In [ ]:
imagenator_non_prod_information = {
    'collection': Collection(id='C1262899916-LARC_CLOUD'),
    'granule_id': 'G1269044683-LARC_CLOUD',
}
associated_output = 'TEMPO_NO2_L2_V03_20240801T195948Z_S012G02_regridded_filtered_product_vertical_column_stratosphere_reformatted'

imagenator_env = {
    Environment.LOCAL: imagenator_non_prod_information,
    Environment.SIT: imagenator_non_prod_information,
    Environment.UAT: imagenator_non_prod_information,
}

if harmony_environment in imagenator_env:
    imagenator_info = imagenator_env[harmony_environment]
else:
    imagenator_info = None

## Test: imagenator L2 request on TEMPO NO2 vertical_column_stratosphere variable

Submit a single-granule request and validate output files.

In [ ]:
if imagenator_info is not None:
    request = Request(
        collection=imagenator_info['collection'],
        granule_id=[imagenator_info['granule_id']],
        variables='product/vertical_column_stratosphere',
        format='image/png',
    )

    job_id = harmony_client.submit(request)
    harmony_client.wait_for_processing(job_id)

    with TemporaryDirectory() as temp_dir:
        output_files = [
            Path(future.result())
            for future in harmony_client.download_all(job_id, directory=temp_dir)
        ]

        assert len(output_files) == 3
        assert output_files[0].suffixes == ['.png']
        assert output_files[1].suffixes == ['.pgw']
        assert output_files[2].suffixes == ['.png', '.aux', '.xml']

        reference_data = Path('reference_data')
        expected_png = reference_data / f'{associated_output}.png'
        expected_pgw = reference_data / f'{associated_output}.pgw'
        expected_aux = reference_data / f'{associated_output}.png.aux.xml'

        assert output_files[0].read_bytes() == expected_png.read_bytes()
        assert output_files[1].read_bytes() == expected_pgw.read_bytes()
        assert output_files[2].read_bytes() == expected_aux.read_bytes()
    print_success('imagenator L2 request on TEMPO NO2 vertical_column_stratosphere')
else:
    print('Skipping test: imagenator L2 not configured for environment')

### Set up environment-dependent variables for imagenator L3:

Define the collection and granule for testing in each environment.

In [ ]:
imagenator_non_prod_information = {
    'collection': Collection(id='C1262899964-LARC_CLOUD'),
    'granule_id': 'G1269043414-LARC_CLOUD',
}
associated_output = 'TEMPO_NO2_L3_V03_20240822T114056Z_S002_filtered_product_vertical_column_stratosphere_reformatted'

imagenator_env = {
    Environment.LOCAL: imagenator_non_prod_information,
    Environment.SIT: imagenator_non_prod_information,
    Environment.UAT: imagenator_non_prod_information,
}

if harmony_environment in imagenator_env:
    imagenator_info = imagenator_env[harmony_environment]
else:
    imagenator_info = None

## Test: imagenator L3 request on TEMPO NO2 vertical_column_stratosphere variable

Submit a single-granule request and validate output files.

In [ ]:
if imagenator_info is not None:
    request = Request(
        collection=imagenator_info['collection'],
        granule_id=[imagenator_info['granule_id']],
        variables='product/vertical_column_stratosphere',
        format='image/png',
    )

    job_id = harmony_client.submit(request)
    harmony_client.wait_for_processing(job_id)

    with TemporaryDirectory() as temp_dir:
        output_files = [
            Path(future.result())
            for future in harmony_client.download_all(job_id, directory=temp_dir)
        ]

        assert len(output_files) == 3
        assert output_files[0].suffixes == ['.png']
        assert output_files[1].suffixes == ['.pgw']
        assert output_files[2].suffixes == ['.png', '.aux', '.xml']

        reference_data = Path('reference_data')
        expected_png = reference_data / f'{associated_output}.png'
        expected_pgw = reference_data / f'{associated_output}.pgw'
        expected_aux = reference_data / f'{associated_output}.png.aux.xml'

        assert output_files[0].read_bytes() == expected_png.read_bytes()
        assert output_files[1].read_bytes() == expected_pgw.read_bytes()
        assert output_files[2].read_bytes() == expected_aux.read_bytes()
    print_success('imagenator L3 request on TEMPO NO2 vertical_column_stratosphere')
else:
    print('Skipping test: imagenator L3 not configured for environment')